In [1]:
import numpy as np
import astropy.constants as c
from simple_slider import get_disk_height

In [2]:
au    = c.au.cgs.value
G     = c.G.cgs.value
k_b   = c.k_B.cgs.value
m_p   = c.m_p.cgs.value
M_sun = c.M_sun.cgs.value
mu    = 2.3

In [3]:
def kanagawa_profile_mask(x, alpha, aspect_ratio, mass_ratio):
    K_prime = mass_ratio**2 * aspect_ratio**-3 / alpha
    K = K_prime / (aspect_ratio**2)
    fact_min_0 = 1 / (1 + 0.04 * K)
    R1 = (fact_min_0 / 4 + 0.08) * K_prime**(1 / 4)
    R2 = 0.33 * K_prime**(1 / 4)
    fact = np.ones_like(x)
    mask = np.abs(x - 1) < R2
    fact[mask] = 4.0 * K_prime**(-1 / 4) * np.abs(x[mask] - 1) - 0.32
    fact[np.abs(x - 1) < R1] = fact_min_0
    return fact

In [4]:
def kanagawa_profile_forloop(x, alpha, aspect_ratio, mass_ratio):
    K_prime = mass_ratio**2 * aspect_ratio**-3 / alpha
    K = K_prime / (aspect_ratio**2)
    fact_min_0 = 1 / (1 + 0.04 * K)
    R1 = (fact_min_0 / 4 + 0.08) * K_prime**(1 / 4)
    R2 = 0.33 * K_prime**(1 / 4)
    fact = np.ones_like(x)
    for i in range(len(x)):
            if np.abs(x[i] - 1) < R1:
                fact[i] = fact_min_0
            elif np.abs(x[i] - 1) < R2:
                fact[i] = 4.0 * K_prime**(-1/4) * np.abs(x[i] - 1) - 0.32
    # sig in units of sig_0
    return fact

In [5]:
r = np.loadtxt("simple_slider_widget/simple_slider/data_1_planet/radius.dat")
x = r / (100*au)
ar = get_disk_height(100*au)/100*au

In [6]:
%timeit kanagawa_profile_mask(x, 1e-3, ar, 4e-4)

17.9 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
%timeit kanagawa_profile_forloop(x, 1e-3, ar, 4e-4)

2.37 ms ± 65.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
